In [ ]:
%run ../../lib/basic-import.ipynb
%run ../../lib/plot-functions.ipynb
%run ../../lib/statistic-functions.ipynb

In [ ]:
relative_dist_mean_std_chau()

In [ ]:
import termtables as tt
import os.path

#fast for finding closest value
from bisect import *

#pip install termtables

In [ ]:
#https://stackoverflow.com/questions/2566412/find-nearest-value-in-numpy-array
import numpy as np
def take_closest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
# Make output folder and error folder
def make_out_folders(ID):
    if not os.path.exists('../data/{}'.format(ID)):
        os.makedirs('../data/{}'.format(ID))

    if os.path.exists('../data/{}/errors'.format(ID)):
        for old_file in listdir('../data/{}/errors'.format(ID)):
            os.remove('../data/{}/errors/'.format(ID)+old_file) #some files are updated instead of deleted - therefor old files need to be removed when rerunning
    else:
        os.makedirs('../data/{}/errors'.format(ID))

# Create file when number of laps are incorrect
def error_number_of_laps(ID, length):
    with open('../data/{}/errors/incorrectlapcount'.format(ID), 'w') as f:
        f.write(str(length))
        f.close()

def error_a_used_for_b(ID,a,b):
    with open('../data/{}/errors/aforb'.format(ID), 'a') as f:
        #values separated by , entries seperated by :
        f.write(str(a)+','+str(b)+':')
        f.close()

def error_mean_for_a(ID,a):
    with open('../data/{}/errors/meanfora'.format(ID), 'a') as f:
        #values separated by , entries seperated by :
        f.write(str(a)+':')
        f.close()

def error_bigbad(ID,a):
    with open('../data/{}/errors/bigbad'.format(ID), 'a') as f:
        f.write(str(a)+':')
        f.close()

def error_nf(ID):
    with open('../data/{}/errors/nf'.format(ID), 'w') as f:
        f.close

In [ ]:
# Returns a distance list with the correct number of laps
# from a list that had too many
def find_best_fitting_lap(ID,laps,mean_dict,std_dict,dist_li,dist_max):
    ret_li = []
    for i in range(laps):
        closeness = std_dict[i]*2
        lap = 'a'

        #Find best fitting lap from actual laps
        for j in range(len(dist_li)):
            temp = abs(mean_dict[i] - dist_li[j] / dist_max)
            if temp < closeness:
                closeness = temp
                lap = j
        # Lap == a -> no good laps, use mean
        if lap == 'a':
            ret_li.append(round(mean_dict[i]*dist_max))
            error_mean_for_a(ID,i)
            lap = i
        # Else, use best fitting lap
        else:
            ret_li.append(dist_li[lap])
        # If pressed lap != lap, give heads up
        if lap != i:
            error_a_used_for_b(ID,lap,i)
    return ret_li

# Use distance list and dataframe to find time list
def get_time_li_from_dist(df,li):
    ret_li = []
    for i in range(len(li)):
        try:
            ret_li.append(df.index[df['distance'] == li[i]])
        except:
            print('something is bad at get li from dist')
            temp_df = df.index[df['distance'] <= li[i]]
            ret_li.append(temp_df.max())
        if type(ret_li[i]) == pd.DatetimeIndex:
            ret_li[i] = pd.Timestamp(ret_li[i][0])
    return ret_li

def new_time_from_dist(df,li):
    ret_li = []
    for i in range(len(li)):
        temp_df = df.index[df['distance'] <= li[i]]
        ret_li.append(temp_df.max())
        if type(ret_li[i]) == pd.DatetimeIndex:
            ret_li[i] = pd.Timestamp(ret_li[i][0])
    return ret_li

# Finds the longest list among any number of lists
def longest_list(*args):
    ret = 0
    for argument in args:
        try:
            if len(argument) > ret:
                ret = len(argument)
        except:
            continue
    return ret

# Finds the longest list and adds every value into a single array
# If one or more lists are out of bounds, an empty string is added instead
def lists_to_single(length, *args):
    ret = []
    for i in range(length):
        temp_li = []
        for argument in args:
            try:
                temp = argument[i]
            except:
                temp = ''
            temp_li.append(temp)
        ret.append(temp_li)
    return ret

In [ ]:
# Find if there is an actual stop in given dataframe. Use progressivly worse checks
# This is necessary as some have no power data. Worst case have been observed in atleast one participant (50712)
# They have power data, but it's never zero - meaning either faulty device or unknown user error
def get_actual_stops(df):
    speed_th = 0.5
    power_th = 1

    # Best - both speed and power under threshold
    ret_df = df[(df['speed'] <= speed_th) & (df['power'] <= power_th)]
    if ret_df.shape[0] != 0:
        return ret_df
    
    # OK - speed under threshold, no power data
    ret_df = df[(df['speed'] <= speed_th) & (df['power'].isnull().all())]
    if ret_df.shape[0] != 0:
        return ret_df

    # Worst - speed under threshold, power data exists, but over threshold
    ret_df = df[(df['speed'] <= speed_th)]
    return ret_df

In [ ]:
# function for plotting fulplot based on 
def pd_plot_full_woglaps(participant_ID, lap_time_li):
    add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
    for participant in participant_files:
        ID = participant[:-4]  

        # this is maybe the dumbest way possible to do this, but was the quickest to come up with
        if str(ID) != str(participant_ID):
            continue

        #for now, we are only looking at 50338
        add_df_338 = add_df[ID]
        # 15
        #Sjekker der alle verdiene ikke er NaNs. 
        #Gjør det samme som din tidligere funksjon, bare med timestamp som index i betraktning :P.
        add_df_338 = add_df_338[~add_df_338.isnull().all(1)]

        #Gjør frekvensen om til rett, slik at det alltid blir timestamp as index
        add_df_338 = add_df_338.asfreq('s')
    
        rightPosition = 1.00
        leftPosition = -0.10

        fig, ax = plt.subplots()

        ax2 = ax.twinx()
        ax3 = ax.twinx()
        ax4 = ax.twinx()
        ax5 = ax.twinx()
        ax6 = ax.twinx()

        ax.autoscale(enable=True, axis='x', tight=True)
        ax2.autoscale(enable=True, axis='x', tight=True)
        ax3.autoscale(enable=True, axis='x', tight=True)
        ax4.autoscale(enable=True, axis='x', tight=True)
        ax5.autoscale(enable=True, axis='x', tight=True)
        ax6.autoscale(enable=True, axis='x', tight=True)


        rightPosition = ax_modification(ax2,'right', rightPosition) - 0.1
        rightPosition = ax_modification(ax3,'right', rightPosition) - 0.1
        leftPosition = ax_modification(ax4,'left', leftPosition) + 0.1
        rightPosition = ax_modification(ax5,'right', rightPosition) - 0.1
        leftPosition = ax_modification(ax6,'left', leftPosition) + 0.1

        add_df_338['accumulated_power'].plot(ax=ax, color='red',legend=False, title="Full race, pd {}".format(ID), figsize=(10,3))
        add_df_338['speed'].plot(ax=ax2, color='blue',legend=False)
        add_df_338['power'].rolling(window=45).mean().plot(ax=ax3, color='green',legend=False)
        #add_df_338['power'].rolling(window=60).mean().plot(ax=ax3, color='green',legend=False) #ser ingen forskjell, går for den med minst datatap
        add_df_338['altitude'].plot(ax=ax4, color='orange',legend=False)
        add_df_338['distance'].plot(ax=ax5, color='lightcoral',legend=False)
        add_df_338['heart_rate'].plot(ax=ax6, color='purple',legend=False)

        for element in lap_time_li:
            ax.axvline((element), color='black', linestyle='--', lw=2)
            #ax.text((element),.5,(element),transform=trans)
            # too much information with text, can be read from table

        trans = ax.get_xaxis_transform()


        ax_style(ax,'Accumulated Power','red')
        ax_style(ax2,'Speed','blue')
        ax_style(ax3,'Power','green')
        ax_style(ax4,'Altitude','orange')
        ax_style(ax5,'Distance','lightcoral')
        ax_style(ax6,'Heartrate','purple')

        y_lim_zero(ax)
        y_lim_zero(ax2)
        y_lim_zero(ax3)
        y_lim_zero(ax4)
        y_lim_zero(ax5)
        y_lim_zero(ax6)


        fig.savefig("fullracepd{}.png".format(ID), format="png", bbox_inches='tight')
        plt.close()

        if not os.path.exists('../data/{}'.format(ID)):
            os.makedirs('../data/{}'.format(ID))
        if os.path.exists('../data/{}/fullracepd{}.png'.format(ID,ID)):
            os.remove('../data/{}/fullracepd{}.png'.format(ID,ID))
        os.rename('./fullracepd{}.png'.format(ID), '../data/{}/fullracepd{}.png'.format(ID,ID))

In [ ]:
def pd_incomplete_race(ID, ti_li, di_li):
    error_nf(ID)
    timestamp_original = ti_li
    timestamp_suggested = []
    timestamp_final = []
    distance_original = di_li
    for i in range(len(timestamp_original)):
        timestamp_suggested.append(timestamp_original[i])
        timestamp_final.append(timestamp_original[i].date())

    allvals_len = longest_list(timestamp_original,timestamp_suggested,timestamp_final,distance_original)
    allVals = lists_to_single(allvals_len,timestamp_original,timestamp_suggested,timestamp_final,distance_original)
    headerTable = ['original timestamp','suggested timestamp','corrected timestamp','original distance']
    dataTableDF = [headerTable] + allVals
    tableDF = pd.DataFrame(dataTableDF[1:],columns=dataTableDF[0]).set_index('original timestamp')
    tableDF.to_csv('correlation_{}'.format(ID)+'.csv')

    if os.path.exists('../data/{}/correlation_{}.csv'.format(ID,ID)):
        os.remove('../data/{}/correlation_{}.csv'.format(ID,ID))
    os.rename('./correlation_{}.csv'.format(ID), '../data/{}/correlation_{}.csv'.format(ID,ID))

    pd_plot_full_woglaps(ID, timestamp_original)

In [ ]:
# Debug
global debugs

def pd_plot_lap_save_corr_non_destructive():
    mean_dict, std_dict = relative_dist_mean_std_chau()
    mean_max_dist, std_max_dist = max_dist_mean_std()
    add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
    
    global debugs

    # Change if different race
    correct_number_of_laps = 4

    # Time befor and after a lap (used when plotting a lap)
    timedelta_after = pd.Timedelta("00:00:30")
    timedelta_before = pd.Timedelta("00:00:15")

    for participant in participant_files:    

        # Get ID and create folders
        ID = participant[:-4]
        #if str(ID) != str(50305):
        #    continue
        make_out_folders(ID)

        # Get dataframe and remove completly empty line from start of dataframe
        participant_df = add_df[ID]
        participant_df = participant_df[~participant_df.isnull().all(1)]
        
        # Find where lap changes value by 1 and make a list of the timestamps and of the distance.
        lapTimestamp_li = list(participant_df.index[participant_df['lap'].diff() == 1])
        lapDistance_li = list(participant_df['distance'][participant_df['lap'].diff() == 1])
        distance_max = participant_df['distance'].max()

        if abs(mean_max_dist - distance_max) > std_max_dist*2:
            #throw error
            print(str(ID)+': NF')
            pd_incomplete_race(ID, lapTimestamp_li, lapDistance_li)
            continue

        # These are used in the csv
        timestamp_original = lapTimestamp_li
        timestamp_suggested = []
        timestamp_final = []
        distance_original = lapDistance_li

        # Check number of laps, create error file if wrong
        if len(lapTimestamp_li) != correct_number_of_laps:
            error_number_of_laps(ID, len(lapTimestamp_li))

        # Find basis for correction
        corr_dist_li = find_best_fitting_lap(ID,correct_number_of_laps, mean_dict, std_dict, lapDistance_li, distance_max)
        corr_time_li = new_time_from_dist(participant_df,corr_dist_li)

        # Now that we have a laplist, we need to see if they are valid BEFORE plotting
        bigbad = [False]
        for i in range(len(corr_time_li)):
            try:
                check_start = participant_df.index.get_loc(corr_time_li[i] - timedelta_before)
            except:
                print(str(ID)+': invalid start - lap '+str(i))
            try:
                check_stopp = participant_df.index.get_loc(corr_time_li[i] + timedelta_after)
            except:
                print(str(ID)+': invalid end - lap '+str(i))
            try:
                check_range = participant_df.iloc[check_start:check_stopp]
            except:
                print(str(ID)+': illegal range - lap '+str(i))
            try:
                temp = get_actual_stops(check_range)
                if temp.shape[0] == 0:
                    timestamp_suggested.append(timestamp_original[i])
                    timestamp_final.append(timestamp_original[i].date())
                    raise ValueError
                else:
                    timestamp_suggested.append(temp.iloc[0].name)
                    timestamp_final.append(temp.iloc[0].name.date())
            except:
                bigbad[0] = True
                bigbad.append(i)

        # If there is no actual stops at the lap, we need to find one, if possible
        # Note that timestamps were still appended to suggested, so it needs to be repaired
        if bigbad[0]:
            debugs.append('bigbad: '+str(bigbad))
            bigbad = bigbad[1:]
            for i in range(len(bigbad)):
                j = bigbad[i]
                try:
                    check_start = participant_df.index.get_loc(timestamp_suggested[j] - timedelta_before)
                    check_stopp = participant_df.index.get_loc(timestamp_suggested[j] + timedelta_after)
                    check_range = participant_df.iloc[check_start:check_stopp]
                    temp = get_actual_stops(check_range)
                    if temp.shape[0] != 0:
                        timestamp_suggested[j] = temp.iloc[0].name
                    else:
                        raise ValueError
                except:
                    print(str(ID)+': bb - maual correction possibly required for lap '+str(j))
                    error_bigbad(ID,j)            

        # List of dataframes, where dataframes contain a range around each lap
        lapRange_df_li = []
        for i in range(correct_number_of_laps):
            try:
                startRow = participant_df.index.get_loc(timestamp_suggested[i]-timedelta_before)
            except:
                print(ID+': startRow failed')
                startRow = participant_df.index.get_loc(corr_time_li[i])
            try:
                endRow = participant_df.index.get_loc(timestamp_suggested[i]+timedelta_after)
            except:
                print(ID+': endRow failed')
                endRow = participant_df.index.get_loc(corr_time_li[i])

            lapRange_df_li.append(participant_df.iloc[startRow:endRow])

        # Create plot for every stop
        for i in range(len(lapRange_df_li)):
            if lapRange_df_li[i].shape[0] == 0:
                print(str(ID)+': bad lap shape')
                continue
            fig, ax = plt.subplots()

            trans = ax.get_xaxis_transform()

            ax2 = ax.twinx()
            ax3 = ax.twinx()
            ax4 = ax.twinx()
            ax5 = ax.twinx()
            ax6 = ax.twinx()

            rightPosition = 1.00
            leftPosition = -0.20

            rightPosition = ax_modification(ax2,'right', rightPosition)
            rightPosition = ax_modification(ax3,'right', rightPosition)
            leftPosition = ax_modification(ax4,'left', leftPosition)
            rightPosition = ax_modification(ax5,'right', rightPosition)
            leftPosition = ax_modification(ax6,'left', leftPosition)
            
            plot_title = "lap {}: -15,+30 pd {}".format(i,ID)

            lapRange_df_li[i]['accumulated_power'].plot(ax=ax, color='red',legend=False, title=plot_title)
            lapRange_df_li[i]['speed'].plot(ax=ax2, color='blue',legend=False)
            lapRange_df_li[i]['power'].plot(ax=ax3, color='green',legend=False)
            lapRange_df_li[i]['altitude'].plot(ax=ax4, color='orange',legend=False)
            lapRange_df_li[i]['distance'].plot(ax=ax5, color='lightcoral',legend=False)
            lapRange_df_li[i]['heart_rate'].plot(ax=ax6, color='purple',legend=False)    
            ax.axvline((timestamp_suggested[i]), color='black', linestyle='--', lw=2)
            ax.text((timestamp_suggested[i]),.5,(timestamp_suggested[i]),transform=trans)

            #dårlige runtime, trenge sikkert ikkje sjekka kver einaste value.
            #bør tenka på ein bedre måte
            check_ylim(ax,lapRange_df_li[i],'accumulated_power')
            check_ylim(ax2,lapRange_df_li[i],'speed')
            check_ylim(ax3,lapRange_df_li[i],'power')
            check_ylim(ax4,lapRange_df_li[i],'altitude')
            check_ylim(ax5,lapRange_df_li[i],'distance')

            y_lim_zero(ax)
            y_lim_zero(ax2)
            y_lim_zero(ax3)
            y_lim_zero(ax4)
            y_lim_zero(ax5)
            y_lim_zero(ax6)

            ax_style(ax,'Accumulated Power','red') #didn't show data
            ax_style(ax2,'Speed','blue')
            ax_style(ax3,'Power','green')
            ax_style(ax4,'Altitude','orange')
            ax_style(ax5,'Distance','lightcoral')
            ax_style(ax6,'Heartrate','purple')
            
            #bbox_inches makes it possible to see all y axis in image.
            #all the y axis didnt show pdf when incrementing rightposition and leftposition.
            #however think we need to resize it to fit in the pdf.
            fig.savefig("../data/{}/lap{}pd{}.png".format(ID,i,ID), format="png", bbox_inches='tight')
            #trying to hide plots from notebook
            plt.close()
            
        #allVals is the values over in one array.
        allvals_len = longest_list(timestamp_original,timestamp_suggested,timestamp_final,distance_original)
        allVals = lists_to_single(allvals_len,timestamp_original,timestamp_suggested,timestamp_final,distance_original)

        # removed lap for now, should add back later
        headerTable = ['original timestamp','suggested timestamp','corrected timestamp','original distance']

        #tableDF, used in csv.
        dataTableDF = [headerTable] + allVals
        tableDF = pd.DataFrame(dataTableDF[1:],columns=dataTableDF[0]).set_index('original timestamp')
        tableDF.to_csv('correlation_{}'.format(ID)+'.csv')

        if os.path.exists('../data/{}/correlation_{}.csv'.format(ID,ID)):
            os.remove('../data/{}/correlation_{}.csv'.format(ID,ID))
        os.rename('./correlation_{}.csv'.format(ID), '../data/{}/correlation_{}.csv'.format(ID,ID))

        # Create the fullplot with lines on original lap button presses
        pd_plot_full_woglaps(ID, timestamp_original)

if False:
    try:
        pd_plot_lap_save_corr_non_destructive()
    except:
        print(debugs)
else:
    pd_plot_lap_save_corr_non_destructive()

In [ ]:
def pd_plot_full():
    add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
    for participant in participant_files:
        ID = participant[:-4]  

        if str(ID) != str(50305):
            continue
        #for now, we are only looking at 50338
        add_df_338 = add_df[ID]
        # 15
        #Sjekker der alle verdiene ikke er NaNs. 
        #Gjør det samme som din tidligere funksjon, bare med timestamp som index i betraktning :P.
        add_df_338 = add_df_338[~add_df_338.isnull().all(1)]

        #Gjør frekvensen om til rett, slik at det alltid blir timestamp as index
        add_df_338 = add_df_338.asfreq('s')
    
        rightPosition = 1.00
        leftPosition = -0.10

        fig, ax = plt.subplots()

        ax2 = ax.twinx()
        ax3 = ax.twinx()
        ax4 = ax.twinx()
        ax5 = ax.twinx()
        ax6 = ax.twinx()

        ax.autoscale(enable=True, axis='x', tight=True)
        ax2.autoscale(enable=True, axis='x', tight=True)
        ax3.autoscale(enable=True, axis='x', tight=True)
        ax4.autoscale(enable=True, axis='x', tight=True)
        ax5.autoscale(enable=True, axis='x', tight=True)
        ax6.autoscale(enable=True, axis='x', tight=True)


        rightPosition = ax_modification(ax2,'right', rightPosition) - 0.1
        rightPosition = ax_modification(ax3,'right', rightPosition) - 0.1
        leftPosition = ax_modification(ax4,'left', leftPosition) + 0.1
        rightPosition = ax_modification(ax5,'right', rightPosition) - 0.1
        leftPosition = ax_modification(ax6,'left', leftPosition) + 0.1

        add_df_338['accumulated_power'].plot(ax=ax, color='red',legend=False, title="Full race, pd {}".format(ID), figsize=(10,3))
        add_df_338['speed'].plot(ax=ax2, color='blue',legend=False)
        add_df_338['power'].rolling(window=45).mean().plot(ax=ax3, color='green',legend=False)
        add_df_338['altitude'].plot(ax=ax4, color='orange',legend=False)
        add_df_338['distance'].plot(ax=ax5, color='lightcoral',legend=False)
        add_df_338['heart_rate'].plot(ax=ax6, color='purple',legend=False)

        trans = ax.get_xaxis_transform()


        ax_style(ax,'Accumulated Power','red')
        ax_style(ax2,'Speed','blue')
        ax_style(ax3,'Power','green')
        ax_style(ax4,'Altitude','orange')
        ax_style(ax5,'Distance','lightcoral')
        ax_style(ax6,'Heartrate','purple')

        y_lim_zero(ax)
        y_lim_zero(ax2)
        y_lim_zero(ax3)
        y_lim_zero(ax4)
        y_lim_zero(ax5)
        y_lim_zero(ax6)


        fig.savefig("fullracepd{}.png".format(ID), format="png", bbox_inches='tight')

        if not os.path.exists('../data/{}'.format(ID)):
            os.makedirs('../data/{}'.format(ID))
        if os.path.exists('../data/{}/fullracepd{}.png'.format(ID,ID)):
            os.remove('../data/{}/fullracepd{}.png'.format(ID,ID))
        os.rename('./fullracepd{}.png'.format(ID), '../data/{}/fullracepd{}.png'.format(ID,ID))

pd_plot_full()

In [ ]:

#def pd_plot_lap_save_corr():
#    add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
#    ID = '50338'
#    rightPosition = 1.00
#    leftPosition = -0.10

#    fig, ax = plt.subplots()
#    fig.subplots_adjust(right=2)

#    ax2 = ax.twinx()
#    ax3 = ax.twinx()
#    ax4 = ax.twinx()
#    ax5 = ax.twinx()
#    ax6 = ax.twinx()

#    rightPosition = ax_modification(ax2,'right', rightPosition)
#    rightPosition = ax_modification(ax3,'right', rightPosition)
#    leftPosition = ax_modification(ax4,'left', leftPosition)
#    rightPosition = ax_modification(ax5,'right', rightPosition)
#    leftPosition = ax_modification(ax6,'left', leftPosition)

#    lapDates = add_df[ID].index[add_df[ID]['lap'].diff()==1]
    #create new pd lap_df. Contains only +30, -15 sec laps, 1 ekstra for nans.
    #lap_df, gir en ny dataframe. Denne inneholder de tre lap_dfs dataframe.
#    lap_dfs = get_lapdf(add_df, ID, 15, 32, lapDates)
#    lap_df = pd.concat(lap_dfs)
#    lap_df.index = map(lambda x: str(x)[-8:], lap_df.index)

#    lap_df['accumulated_power'].plot(ax=ax, color='red',legend=False, title="all laps -15,+30 pd 50338")
#    lap_df['speed'].plot(ax=ax2, color='blue',legend=False)
#    lap_df['power'].plot(ax=ax3, color='green',legend=False)
#    lap_df['altitude'].plot(ax=ax4, color='orange',legend=False)
#    lap_df['distance'].plot(ax=ax5, color='lightcoral',legend=False)
#    lap_df['heart_rate'].plot(ax=ax6, color='purple',legend=False)

#    ax_style(ax,'Accumulated Power','red')
#    ax_style(ax2,'Speed','blue')
#    ax_style(ax3,'Power','green')
#    ax_style(ax4,'Altitude','orange')
#    ax_style(ax5,'Distance','lightcoral')
#    ax_style(ax6,'Heartrate','purple')

    #getting correlation values, now values and changeable values.
#    nowVals = get_now_val(lapDates, ID, add_df)
#    corrVals = get_corr_val(lap_dfs)
#    changVals = [date.date() for date in lapDates]

    #allVals is the values over in one array.
#    allVals = []
#    for i in range(len(corrVals)):
#        allVals.append([changVals[i]] + [corrVals[i]] + nowVals[i])

    #table which is shown. might change the names.
#    headerTable = ['timestamp correlation changeable','timestamp correlation suggestion','timestamp now','lap','distance']
#    print(tt.to_string(allVals, header=headerTable))

    #tableDF, used in csv.
#    dataTableDF = [headerTable] + allVals
#    tableDF = pd.DataFrame(dataTableDF[1:],columns=dataTableDF[0]).set_index('timestamp correlation changeable')
#   tableDF.to_csv('correlation_{}'.format(ID)+'.csv')


#pd_plot_lap_save_corr()


In [ ]:
add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
def pd_plot_corr():
    for participant in participant_files:
        ID = participant[:-4]
        add_df_338_new = add_df[ID]
        add_df_338_new = add_df_338_new[~add_df_338_new.isnull().all(1)]

        corrDF = pd.read_csv('../data/{}/correlation_{}.csv'.format(ID,ID)).set_index("corrected timestamp")
        corrTS = get_corrTS(corrDF)

        lapRange_df_li = []
        timedelta_30s = pd.Timedelta("00:00:30")
        timedelta_15s = pd.Timedelta("00:00:15")
        for i in range(len(corrTS)):
            startRow = add_df_338_new.index.get_loc(corrTS[i]-timedelta_15s)
            endRow = add_df_338_new.index.get_loc(corrTS[i]+timedelta_30s)
            lapRange_df_li.append(add_df_338_new.iloc[startRow:endRow])

        for i in range(len(lapRange_df_li)):
            rightPosition = 1.00
            leftPosition = -0.20

            fig, ax = plt.subplots()

            ax2 = ax.twinx()
            ax3 = ax.twinx()
            ax4 = ax.twinx()
            ax5 = ax.twinx()
            ax6 = ax.twinx()

            rightPosition = ax_modification(ax2,'right', rightPosition)
            rightPosition = ax_modification(ax3,'right', rightPosition)
            leftPosition = ax_modification(ax4,'left', leftPosition)
            rightPosition = ax_modification(ax5,'right', rightPosition)
            leftPosition = ax_modification(ax6,'left', leftPosition)

            lapRange_df_li[i]['accumulated_power'].plot(ax=ax, color='red',legend=False, title="corr lap {} -15,+30 pd {}".format(i, ID))
            lapRange_df_li[i]['speed'].plot(ax=ax2, color='blue',legend=False)
            lapRange_df_li[i]['power'].plot(ax=ax3, color='green',legend=False)
            lapRange_df_li[i]['altitude'].plot(ax=ax4, color='orange',legend=False)
            lapRange_df_li[i]['distance'].plot(ax=ax5, color='lightcoral',legend=False)
            lapRange_df_li[i]['heart_rate'].plot(ax=ax6, color='purple',legend=False)

            trans = ax.get_xaxis_transform()
            ax.axvline((corrTS[i]), color='black', linestyle='--', lw=2)
            ax.text((corrTS[i]),.5,(corrTS[i]),transform=trans)

            check_ylim(ax,lapRange_df_li[i],'accumulated_power')
            check_ylim(ax2,lapRange_df_li[i],'speed')
            check_ylim(ax3,lapRange_df_li[i],'power')
            check_ylim(ax4,lapRange_df_li[i],'altitude')
            check_ylim(ax5,lapRange_df_li[i],'distance')

            ax_style(ax,'Accumulated Power','red')
            ax_style(ax2,'Speed','blue')
            ax_style(ax3,'Power','green')
            ax_style(ax4,'Altitude','orange')
            ax_style(ax5,'Distance','lightcoral')
            ax_style(ax6,'Heartrate','purple')

# pd_plot_corr()